#Actividad 3:Mineria de Datos

*Carlos Eduardo Inca Cruz*


El algoritmo a priori es un algoritmo utilizado en minería de datos, sobre bases de datos transaccionales, que permite encontrar de forma eficiente "conjuntos de ítems frecuentes", los cuales sirven de base para generar reglas de asociación. Procede identificando los ítems individuales frecuentes en la base y extendiéndolos a conjuntos de mayor tamaño siempre y cuando esos conjuntos de datos aparezcan suficientemente seguidos en dicha base de datos. Este algoritmo se ha aplicado grandemente en el análisis de transacciones comerciales y en problemas de predicción.

La minería de reglas de asociación consta de 2 pasos:



*   Encuentra todos los conjuntos de elementos frecuentes.
*   Genere reglas de asociación a partir de los conjuntos de elementos frecuentes anteriores.

#Importamos librerias

In [35]:
import itertools
import numpy as np
import pandas as pd 
from tabulate import tabulate 

#Implementar el algoritmo Apriori

In [214]:


# Funciones para calcular y extraer reglas de asociación de una frecuencia determinada
# conjunto de elementos generado por el algoritmo Apriori.


#Genera conjuntos de elementos candidatos
def GenerarCandidatosIS(Frec_ISets, k):
    # lista de conjuntos de elementos frecuentes fusionados
    Elementos_Retirados = []
    # lista de conjuntos de elementos frecuentes fusionados
    lenLk = len(Frec_ISets) 
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            a=list(Frec_ISets[i])
            b=list(Frec_ISets[j])
            a.sort()
            b.sort()
            # primeros k-2 elementos de Frec_ISets [i]
            F1 = a[:k-2] 
            # # primeros k-2 elementos de Frec_ISets [i]
            F2 = b[:k-2] 

            if F1 == F2: 
                
                # Mezclar los conjuntos de elementos frecuentes.
                Elementos_Retirados.append(Frec_ISets[i] | Frec_ISets[j])

    return Elementos_Retirados
#Genera un conjunto de reglas candidatas
def Gen_Reglas_Candidatas(Elem_Frecuentes, H, Datos_De_Soporte, Reglas, Min_Conf=0.5, Bandera=False):

    m = len(H[0])
    if m == 1:
        Hmp1 = Calcular_Confianza(Elem_Frecuentes, H, Datos_De_Soporte, Reglas, Min_Conf, Bandera)
    if (len(Elem_Frecuentes) > (m+1)):
        Hmp1 = GenerarCandidatosIS(H, m+1) # generar conjuntos de elementos candidatos
        Hmp1 = Calcular_Confianza(Elem_Frecuentes, Hmp1, Datos_De_Soporte, Reglas, Min_Conf, Bandera)
        if len(Hmp1) > 1:
            # Si hay reglas candidatas por encima del mínimo de confianza
            # umbral, recurre en la lista de estas reglas candidatas.
            Gen_Reglas_Candidatas(Elem_Frecuentes, Hmp1, Datos_De_Soporte, Reglas, Min_Conf, Bandera)
#Para calcular la confianza, iteramos a través de los conjuntos de elementos frecuentes ydatos de soporte asociados. 
def Calcular_Confianza(Elem_Frecuentes, H, Datos_De_Soporte, Reglas, Min_Conf=0.5, Bandera=False):
    Total_Reglas=[]
    Reglas_Aceptadas = [] #lista de candidatos Reglas por encima del umbral mínimo de confianza
    
    for conseq in H: # iterate over the frequent itemsets
        conf = Datos_De_Soporte[Elem_Frecuentes] / Datos_De_Soporte[Elem_Frecuentes - conseq]
        if conf >= Min_Conf:
            Reglas.append((Elem_Frecuentes - conseq, conseq, conf))
            Reglas_Aceptadas.append(conseq)
            if Bandera:

                Total_Reglas.append(("".join([str(i) + ", " for i in iter(Elem_Frecuentes-conseq)]).rstrip(', '),"".join([str(i) + ", " for i in iter(conseq)]).rstrip(', '),str(round(conf, 3)), str(round(Datos_De_Soporte[Elem_Frecuentes], 3))))
                #Imprimir las reglas seleccionadas
                print("" + "Ant =" "".join([str(i) + ", " for i in iter(Elem_Frecuentes-conseq)]).rstrip(', ') + " ---> " + "Cons="  + "".join([str(i) + ", " for i in iter(conseq)]).rstrip(', ')  + ":  conf = " + str(round(conf, 3)) + ", sup = " + str(round(Datos_De_Soporte[Elem_Frecuentes], 3)))
                
    return Reglas_Aceptadas

# Genera un conjunto de reglas candidatas
def generate_Reglas(F, Datos_De_Soporte, Min_Conf=0.5, Bandera=True):
   
    Reglas = []
    for i in range(1, len(F)):
        for Elem_Frecuentes in F[i]:
            H1 = [frozenset([itemset]) for itemset in Elem_Frecuentes]
            if (i > 1):
                Gen_Reglas_Candidatas(Elem_Frecuentes, H1, Datos_De_Soporte, Reglas, Min_Conf, Bandera)
            else:
                Calcular_Confianza(Elem_Frecuentes, H1, Datos_De_Soporte, Reglas, Min_Conf, Bandera)

    return Reglas





def generate_association_rules(dataset,Min_Sup=0.6, Bandera=True):
    # lista de todos los elementos en la base de datos de transacciones
    c1 = [] 
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    ListaCandidatos=list(map(frozenset, c1))
    
    # Elimine los conjuntos de 1 elemento candidatos mediante la poda basada en soporte para generar conjuntos de 1 elemento frecuentes.
    # establecido para conteos de Suport
    sc = {} 
    for tid in dataset:
        for can in ListaCandidatos:
            if can.issubset(tid):
                sc.setdefault(can, 0)
                sc[can] += 1
     # número total de transacciones en el conjunto de datos
    Numero_De_Elementos = float(len(dataset))
    # matriz para conjuntos de elementos sin podar
    Elementos_Retirados = [] 
    # conjunto de datos de soporte para conjuntos de elementos correspondientes
    Datos_De_Soporte = {} 
    for key in sc:
        # Calcule el soporte de la clave del conjunto de elementos.
        support = sc[key] / Numero_De_Elementos
        if support >= Min_Sup:
            Elementos_Retirados.insert(0, key)
        Datos_De_Soporte[key] = support
    F1, Datos_De_Soporte = Elementos_Retirados, Datos_De_Soporte

     # lista de todos los elementos en la base de datos de transacciones
    c1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    C1=list(map(frozenset, c1))
    D = list(map(set, dataset))
    # establecido para conteos de Suport
    sc = {} 
    for tid in dataset:
        for can in C1:
            if can.issubset(tid):
                sc.setdefault(can, 0)
                sc[can] += 1
    # número total de transacciones en el conjunto de datos
    Numero_De_Elementos = float(len(dataset)) 
    # matriz para conjuntos de elementos sin podar
    Elementos_Retirados = [] 
    # conjunto de datos de soporte para conjuntos de elementos correspondientes
    Datos_De_Soporte = {}
    for key in sc:
        # Calcule el soporte de la clave del conjunto de elementos.
        support = sc[key] / Numero_De_Elementos
        if support >= Min_Sup:
            Elementos_Retirados.insert(0, key)
        Datos_De_Soporte[key] = support
    # podar candidato 1-itemsets
    F1, Datos_De_Soporte = Elementos_Retirados, Datos_De_Soporte 
    # lista de conjuntos de elementos frecuentes; inicializado a conjuntos frecuentes de 1 itemsets
    F = [F1] 
    k = 2 
    # la cardinalidad del conjunto de elementos
    while (len(F[k - 2]) > 0):
        # generar conjuntos de elementos candidatos
        Ck = GenerarCandidatosIS(F[k-2], k) 


        # podar itemsets candidatos
        Fk, supK = Filtrar_Support(D, Ck, Min_Sup) 
        # actualice los conteos de soporte para reflejar la poda
        Datos_De_Soporte.update(supK)
        # agregue los conjuntos de elementos candidatos podados a la lista de conjuntos de elementos frecuentes
        F.append(Fk) 
        k += 1

    return F,Datos_De_Soporte



#Importamos datos

Importamos spotify.npy el cual contiene playlists escuchadas por los usuarios

Ahora importamos los datos del archivo 

```
spotify.npy
```
usamos subarreglos por el alto costo computacional del programa y usamos el los modulos 
$$O(n)=2^{[S]}$$



In [215]:
Nombre_Archivo = 'spotify.npy'
dataSet = np.load(Nombre_Archivo, allow_pickle=True)
dataSet=list(dataSet.tolist().values())[21710:21740]
dataSet=[i[0:5] for i in dataSet]

#Programa Principal
#lista de transacciones; cada transacción es una lista de artículos
dataset = load_dataset() 
# conjunto de transacciones; cada transacción es una lista de artículo
D = list(map(set, dataset)) 


Frecuent_Item_Sets, Datos_De_Soporte = generate_association_rules(dataset,Min_Sup=0.6, Bandera=True)
# Genere las reglas de asociación a partir de una lista de conjuntos de elementos frecuentes.
H = generate_Reglas(Frecuent_Item_Sets, Datos_De_Soporte, Min_Conf=0.8, Bandera=True)

Fathoms Below - Broadway Cast Recording ---> Cons=Daughters Of Triton - Broadway Cast Recording:  conf = 1.0, sup = 0.8
Daughters Of Triton - Broadway Cast Recording ---> Cons=Fathoms Below - Broadway Cast Recording:  conf = 1.0, sup = 0.8
Daughters Of Triton - Broadway Cast Recording ---> Cons=The World Above - Broadway Cast Recording:  conf = 1.0, sup = 0.8
The World Above - Broadway Cast Recording ---> Cons=Daughters Of Triton - Broadway Cast Recording:  conf = 1.0, sup = 0.8
Fathoms Below - Broadway Cast Recording ---> Cons=The World Above - Broadway Cast Recording:  conf = 1.0, sup = 0.8
The World Above - Broadway Cast Recording ---> Cons=Fathoms Below - Broadway Cast Recording:  conf = 1.0, sup = 0.8
Daughters Of Triton - Broadway Cast Recording, Ant =Fathoms Below - Broadway Cast Recording ---> Cons=The World Above - Broadway Cast Recording:  conf = 1.0, sup = 0.8
The World Above - Broadway Cast Recording, Ant =Fathoms Below - Broadway Cast Recording ---> Cons=Daughters Of Trito

In [216]:
Nombre_Archivo = 'spotify.npy'
dataSet = np.load(Nombre_Archivo, allow_pickle=True)
dataSet=list(dataSet.tolist().values())[11700:11730]
dataSet=[i[0:5] for i in dataSet]
dataSet

#Programa Principal
#lista de transacciones; cada transacción es una lista de artículos
dataset = load_dataset() 
# conjunto de transacciones; cada transacción es una lista de artículo
D = list(map(set, dataset)) 


Frecuent_Item_Sets, Datos_De_Soporte = generate_association_rules(dataset,Min_Sup=0.6, Bandera=True)
# Genere las reglas de asociación a partir de una lista de conjuntos de elementos frecuentes.
H = generate_Reglas(Frecuent_Item_Sets, Datos_De_Soporte, Min_Conf=0.8, Bandera=True)

Fathoms Below - Broadway Cast Recording ---> Cons=Daughters Of Triton - Broadway Cast Recording:  conf = 1.0, sup = 0.8
Daughters Of Triton - Broadway Cast Recording ---> Cons=Fathoms Below - Broadway Cast Recording:  conf = 1.0, sup = 0.8
Daughters Of Triton - Broadway Cast Recording ---> Cons=The World Above - Broadway Cast Recording:  conf = 1.0, sup = 0.8
The World Above - Broadway Cast Recording ---> Cons=Daughters Of Triton - Broadway Cast Recording:  conf = 1.0, sup = 0.8
Fathoms Below - Broadway Cast Recording ---> Cons=The World Above - Broadway Cast Recording:  conf = 1.0, sup = 0.8
The World Above - Broadway Cast Recording ---> Cons=Fathoms Below - Broadway Cast Recording:  conf = 1.0, sup = 0.8
Daughters Of Triton - Broadway Cast Recording, Ant =Fathoms Below - Broadway Cast Recording ---> Cons=The World Above - Broadway Cast Recording:  conf = 1.0, sup = 0.8
The World Above - Broadway Cast Recording, Ant =Fathoms Below - Broadway Cast Recording ---> Cons=Daughters Of Trito

#Explicar las reglas obtenidas

Los resultados obtenidos nos muestran que varias de esas canciones(Antecedentes) estan relacionadas con otras(Consecuentes) por lo cual se debe agrupar dichas canciones dentro de un solo grupo para que la aplicacion de musica sea mas amigable

Analizamos 2 reglas aleatoriamente

Analisis de regla 1

> Ant =The World Above - Broadway Cast Recording ---> Cons=Fathoms Below - Broadway Cast Recording:  conf = 1.0, sup = 0.8



1.   Ambas son canciones para niños
2.   Ambas pertenecen a peliculas animadas de disney y son recitales
3.   Ambas estan en ingles
4.   El 'soporte' de un conjunto de items X en una base de datos C se define como la proporción de transacciones en la base de datos que contiene dicho conjunto de items EL numero 1 nos indica que esa cancion esta en el total de las playlists.
5.   La regla 


```
Ant =The World Above - Broadway Cast Recording--->Cons=Fathoms Below - Broadway Cast Recording: nso*

```
se encuentra en el 80% de las muestras







Analisis de regla 2

> Ant =Daughters Of Triton - Broadway Cast Recording, Circles ---> Cons=The World Above - Broadway Cast Recording, Fathoms Below - Broadway Cast Recording:  conf = 1.0, sup = 0.6



1.   Ambas son recitales de Broadway
2.   Ambas estan en ingles
3.   El 'soporte' de un conjunto de items X en una base de datos C se define como la proporción de transacciones en la base de datos que contiene dicho conjunto de items EL numero 1 nos indica que esa cancion esta en el total de las playlists.
4.   La regla 


```
Ant =Daughters Of Triton - Broadway Cast Recording, Circles ---> Cons=The World Above - Broadway Cast Recording, Fathoms Below - Broadway Cast Recording:  conf = 1.0, sup = 0.6

```
se encuentra en el 80% de las muestras